# Surface molecule numbers compendium
Counts of TCR, pMHC as a function of antigen pulse concentration, tumor antigens, etc. on each tumor and T cell line for which I have data. 

All calibrations (except the oldest ones) were done with Bangs Labs kits, which tend to have higher counts than BD kits. As long as we are consistent, that's fine. 

Using the concatenated dataframe from Sooraj. Simply computing the geometric mean of each molecule on each cell type; we don't need more than that. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit, lsq_linear
from scipy.stats import linregress
import os

# Local modules
import sys
if "../" not in sys.path:
    sys.path.insert(1, "../")
from utils.preprocess import geo_mean_apply

In [2]:
data_folder = "../data/surface_counts/"
do_save_stats = False

## Useful functions

In [3]:
def michaelis_menten(x, a, h):
    normx = (x / h)
    return a * normx / (1.0 + normx)

def loglog_michaelis_menten(logx, a, h):
    logh = np.log(h)
    return np.log(a * np.exp(logx - logh) / (1.0 + np.exp(logx - logh)))

def loglog_michaelis_background(logx, a, h, back):
    logh = np.log(h)
    # a is the real amplitude, there is a background on top, additive. 
    return np.log(a*np.exp(logx - logh) / (1.0 + np.exp(logx - logh)) + back)

def bootstrap_f_ci(x, f, a=0.05, reps=100, rel_size=0.8, rgen=None):
    # Generate reps bootstrap samples, compute f of each, compute 95 % CI of f
    # f could be geometric mean for instance. 
    if rgen is None:
        rgen = np.random.default_rng()
    boot_f_samples = []
    for i in range(reps):
        bootsamp = rgen.choice(x, size=int(rel_size*x.size), replace=True)
        boot_f_samples.append(f(bootsamp))
    # Confidence intervals
    boot_f_samples = np.asarray(boot_f_samples)
    ci_bounds = np.quantile(boot_f_samples, [a/2, 1.0-a/2])
    labels = ["CI "+str(round(x, 3)) for x in [a/2, 1.0-a/2]]
    return {labels[i]:ci_bounds[i] for i in range(2)}

def geo_mean(x):
    return np.exp(np.log(x).mean())

def stdev_of_log10(x):
    return np.std(np.log10(x), ddof=1)

def stats_from_single_cells(ser, by, seed=None, boot_kws={}):
    boot_kws.setdefault("a", 0.05)
    boot_kws.setdefault("reps", 1000)
    boot_kws.setdefault("rel_size", 0.8)
    ser_stats = (ser.groupby(by).aggregate(**{
                    "Mean": "mean", 
                    "Geometric mean": geo_mean,
                    "Percentile 2.5": lambda x: np.quantile(x, 0.025), 
                    "Percentile 97.5": lambda x: np.quantile(x, 0.975), 
                    "Median": "median", 
                    "Standard deviation": "std",
                    "Stdev log10": stdev_of_log10, 
                    "dof": "count"
                }))
    rgen = np.random.default_rng(seed)
    # CI on mean
    new_cols = ["Mean CI 0.025", "Mean CI 0.975"]
    ser_stats[new_cols] = (ser.groupby(by).apply(
                            bootstrap_f_ci, np.mean, 
                            **boot_kws, rgen=rgen).unstack(-1))
    # CI on geometric mean
    new_cols = ["Geometric mean CI 0.025", "Geometric mean CI 0.975"]
    ser_stats[new_cols] = (ser.groupby(by).apply(
                            bootstrap_f_ci, geo_mean, 
                            **boot_kws, rgen=rgen).unstack(-1))
    return ser_stats

## MHC loading vs pulse concentration (from RMA-S cells)
How do pulse concentrations relate to loaded pMHCs? The pulse vs pMHC calibration was done on B6 splenocytes, not on all tumor cells, for which we only know the total number of MHCs available for binding. Can we infer ligand numbers at all pulse concentrations for the latter?

Well, from a simple ligand-receptor binding model, where ligand=antigen and receptor=pMHC, the number of bound receptors $M$, as a function of ligand concentration $L$, obeys:

$$ \dot{M} = \kappa (M_{tot} - M) L - \lambda M $$

with steady-state 

$$ M = M_{tot} \frac{L}{L + K_D}  \quad \text{where } K_D = \lambda/\kappa $$

The $K_D$ is the point at which receptors start to saturate, and it depends only on the affinity (binding and unbinding rates) of the antigen-MHC pair, *not* on the total number of receptors. So, $K_D$ can be assumed constant across APC types (at least for SIINFEKL peptides); changing the total number of MHC on APCs simply scales $M$ proportionally, while the $L$ concentration saturating the $M_{tot}$ receptors does not change. So for instance, 1 $\mu$ M pulses will always saturate MHCs, no matter if the APC has $10^4$ or $10^6$ MHCs on its surface.   

In fact, we can fit $K_D$ on MFI data, then scale to absolute receptor numbers just using the total MHC count, available for tumor cells at least. 


### Final word on how to convert pulse concentration to pMHC number
In the end, all we need to store is the pulse-loading $K_D$, fitted once, and the total number of MHC available on each cell type (converted from MFI measurement). WE will do the latter later; here let's just infer the loading $K_D$ from the RMAs assays. 

### Fitting Michaelis-Menten relationship

For an actual fit, we need to fit the background as well, and to force the amplitude and background around the actual values we see. Otherwise, can't fit a MM function on data that starts at $10^3$! The right model to fit is MM + background. 

In [ ]:
df_rmas = pd.read_hdf(data_folder + "combinedRMASDf_MFI.hdf", key="df")
# Convert concentrations to uM, more convenient to read. 
df_rmas = df_rmas.rename(lambda x: x*1e6, level="Concentration_M")
# Reorder levels to more convenient form
df_rmas = df_rmas.reorder_levels(["Experiment", "Replicate", "Peptide", "Concentration_M"])
df_rmas.index = df_rmas.index.set_names(["Experiment", "Replicate", "Peptide", "Concentration_uM"])
df_rmas

For every peptide, every replicate, fit Michaelis-Menten $R = R_0 L / (L + K_D)$, but fit in log-log scale: if $y=\log(R)$ and $x=\log(L)$, fit $y$ vs $x$ in the following relationship: $y = \log\left(\frac{R_0  e^x}{e^x + K_D} \right)$. 
So MM parameters describe relationship in linear scale, we just fitted them to minimize error on log-scale to ensure multiple orders of magnitude are well captured. This is a linear MM relationship plotted in log-log, not a MM function fitted on the log-log plot. 

In [5]:
def fit_ligand_receptor_mm(df):
    """ To be applied on the sub-dataframe corresponding to one dose-response. """
    x = np.log(df.index.get_level_values("Concentration_uM").values)
    y = df.values.flatten()
    pbounds = np.asarray([(0.7*np.amax(y), np.exp(x.min())/10.0, 0.5*np.amin(y)), 
               (2.0*np.amax(y), np.exp(x.max())*10, 1.5*np.amin(y))])
    y = np.log(y)
    l_conc_mm_params = curve_fit(loglog_michaelis_background, x, y, p0=np.mean(pbounds, axis=0), bounds=pbounds)[0]
    return l_conc_mm_params

In [ ]:
all_params = []
gby = df_rmas.groupby(["Experiment", "Replicate", "Peptide"])
for lbl, gp in gby:
    fit_params = fit_ligand_receptor_mm(gp)
    all_params.append(fit_params)
all_params = pd.DataFrame(all_params, index=pd.MultiIndex.from_tuples(gby.groups, 
                names=["Experiment", "Replicate", "Peptide"]), 
                columns=["amplitude", "ec50", "background"])
all_params = all_params.reorder_levels(["Experiment", "Replicate", "Peptide"])
all_params

In [ ]:
# Plot fits to make sure they are reasonable
plot_idx = all_params.index.droplevel(["Peptide"]).unique()
n_plots = len(plot_idx)
fig, axes = plt.subplots(1, n_plots)
fig.set_size_inches(10, 3.5)
axes = axes.flatten()
all_params = all_params.sort_index()
df_rmas = df_rmas.sort_index()
df_curves = {}
for i, k in enumerate(plot_idx):
    for pep in all_params.loc[k].index.unique():
        x = df_rmas.loc[k+(pep,)].index.get_level_values("Concentration_uM").values
        xrange = np.geomspace(x.min(), x.max(), 101)
        y = df_rmas.loc[k+(pep,)].values.flatten()
        pms = all_params.loc[k+(pep,)]
        ymod = np.exp(loglog_michaelis_background(np.log(xrange), *pms))
        xy_df = pd.DataFrame(np.stack([xrange, ymod], axis=1), 
                             index=pd.RangeIndex(101, name="Point"), 
                             columns=["x", "y"])
        df_curves[tuple(k) + (pep,)] = xy_df.copy()
        li, = axes[i].plot(xrange, ymod, label=pep)
        c = li.get_color()
        axes[i].plot(x, y, color=c, mfc=c, mec=c, marker="o", ms=6, ls="none")
        axes[i].axvline(pms[1], ls="--", lw=1.0, color=c)
    axes[i].set_title(k)
    axes[i].set(xlabel="Concentration (uM)", ylabel="MFI", yscale="log", xscale="log")

df_curves = pd.concat(df_curves, names=all_params.index.names)
df_curves

fig.tight_layout()
plt.show()
plt.close()

In [ ]:
#@title Statistics of loading EC50s
randomgen = np.random.default_rng(0x339788df8e03ab6f23168dec3dd313f3)
booted_params = np.log(all_params.T.copy())
booted_params = booted_params.apply(bootstrap_f_ci, axis=1, raw=False, args=(np.mean,), 
                        a=0.05, reps=1000, rel_size=0.8, rgen=randomgen, result_type="expand")
# Add mean
booted_params = np.exp(booted_params)
booted_params["Geometric mean"] = all_params.T.apply(geo_mean, axis=1)

# We only need the K_D; amplitude and background are relative to specific MFI here
# And will be replaced by the number of MHC on each APC type. 
mhc_pulse_kd = booted_params.loc["ec50"]
mhc_pulse_kd.name = "K_D (uM)"

# Add 95 % CI of the distribution (not just CI of mean)
booted_lower_ci = np.log(all_params.T).apply(bootstrap_f_ci, axis=1, raw=False, 
                                             args=(lambda x: np.quantile(x, 0.025),), 
                        a=0.05, reps=1000, rel_size=0.8, rgen=randomgen, result_type="expand")
booted_upper_ci = np.log(all_params.T).apply(bootstrap_f_ci, axis=1, raw=False, 
                                             args=(lambda x: np.quantile(x, 0.975),), 
                        a=0.05, reps=1000, rel_size=0.8, rgen=randomgen, result_type="expand")
mhc_pulse_kd.loc["Quantile 0.025"] = np.exp(booted_lower_ci.loc["ec50"].mean())
mhc_pulse_kd.loc["Quantile 0.975"] = np.exp(booted_upper_ci.loc["ec50"].mean())
mhc_pulse_kd.loc["dof"] = all_params["ec50"].count()
mhc_pulse_kd  # in uM

## All surface molecule numbers
Includes:
- MHC numbers (to combine with fraction of loaded MHC at a given pulse concentration)
- CAR antigens numbers
- TCR numbers
- CAR numbers

In [ ]:
full_file = "completeMoleculeNumberDf.hdf"
df = pd.read_hdf(data_folder + full_file)
df

In [ ]:
df_surface_numbers_stats = stats_from_single_cells(df["Molecules (#)"], by=["Cell", "Marker"], 
                                    seed=0xfdcef4d3def776989b198568b5886be6)
df_surface_numbers_stats

In [ ]:
# Check that the numbers agree well with what I had before
df_surface_numbers_stats["Geometric mean"].loc[(["BEAS2B", "PC9"], "MHC")]

## Save results to one hdf file with both dataframes

In [12]:
filename = data_folder + "surface_molecule_summary_stats.h5"
useful_frames = {
    "mhc_pulse_kd": mhc_pulse_kd, 
    "surface_numbers_stats": df_surface_numbers_stats, 
}
if do_save_stats:
    for nm, df in useful_frames.items():
        df.to_hdf(filename, key=nm)

In [13]:
# Export to LateX for manuscript: this is now done by the dedicated Python script
# secondary_scripts/mcmc_ci_to_latex.py. 
# secondary_scripts/print_dataset_sizes.py also prints 
# important information for the Supplementary Tables. 

In [ ]:
# Also export MHC pulse KD to a small tex string to be inserted
# Round to significant figure
pulse_kd_tex = np.log10(1e3*mhc_pulse_kd[["Geometric mean", "Quantile 0.975", "Quantile 0.025"]])
pulse_kd_tex.index = pd.Index(["m", "up", "lo"])
# Find last significant digit
decimal_place = min(abs(pulse_kd_tex["up"] - pulse_kd_tex["m"]), 
                      abs(pulse_kd_tex["m"] - pulse_kd_tex["lo"]))
decimal_place = abs(np.floor(np.log10(decimal_place)).astype(int))



kd_load_str = (r"$\log_{10} (K_{\mathrm{D,\, load}}/1\mathrm{nM}) = " 
            + str(round(pulse_kd_tex["m"], decimal_place))
            + r"^{+" 
            + str(round(pulse_kd_tex["up"] - pulse_kd_tex["m"], decimal_place)) 
            + r"}_{-"
            + str(round(pulse_kd_tex["m"] - pulse_kd_tex["lo"], decimal_place))
            + r"}$"
)
print(kd_load_str)
if do_save_stats:
    with open("../results/for_plots/mhc_pulse_kd_string.tex", "w") as f:
        f.write(kd_load_str)